<a href="https://colab.research.google.com/github/KarthikaRajagopal44/rag-kg/blob/main/Copy_of_rag_kg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index llama-index-llms-groq groq llama-index-embeddings-huggingface ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56

In [ ]:
pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 49.0 MB/s eta 0:00:00


In [ ]:
pip install --upgrade llama_index


In [ ]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00


In [ ]:

from llama_index.core import VectorStoreIndex, Document  # Import the Document class
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.core.node_parser import SentenceSplitter
from IPython.display import display
import ipywidgets as widgets
import os
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

# Set the API key for Groq LLM
os.environ["GROQ_API_KEY"] = "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"  # Update with your actual Groq API key

# Create your embedding model and LLM manually (Groq)
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = Groq(model="llama3-70b-8192", api_key=os.getenv("GROQ_API_KEY"))

# Define the prompt template for generating the answers
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

# Define the context for your prompt
context = "This directory contains multiple documents providing examples and solutions for various programming tasks."

# Data ingestion: load a single PDF file using PyPDF2
file_path = "/content/Manual example - AeroCraft ACE-900.pdf"  # Make sure this is the correct path

# Function to read PDF file and extract text
def extract_pdf_text(file_path):
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# Extract text from the PDF
pdf_text = extract_pdf_text(file_path)

# Wrap the extracted text into a Document object
documents = [Document(text=pdf_text)]

# Split the documents into nodes using SentenceSplitter
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents)  # Pass Document objects to SentenceSplitter

# Create and persist the vector store index
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, llm=llm)
vector_index.storage_context.persist(persist_dir="./storage_mini")

# Load the index from storage
storage_context = vector_index.storage_context
index = vector_index

# Create the interactive widgets for the user input
input_box = widgets.Text(
    value='Explain Python?',  # Default question
    placeholder='Type your question here',
    description='Question:',
    disabled=False
)

output_area = widgets.Output()

# Define the button click handler
# def on_button_click(b):
#     with output_area:
#         output_area.clear_output()
#         question = input_box.value
#         query_prompt = prompt_template.format(context=context, question=question)
#         resp = index.as_query_engine().query(query_prompt)
#         print(resp.response)
def on_button_click(b):
    with output_area:
        output_area.clear_output()
        question = input_box.value
        query_prompt = prompt_template.format(context=context, question=question)
        # Explicitly pass the LLM here
        resp = index.as_query_engine(llm=llm).query(query_prompt)
        print(resp.response)


# Create the button widget
button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

# Link the button click to the handler
button.on_click(on_button_click)

# Display the widgets
display(input_box, button, output_area)

# Now, the query engine is ready to accept questions


Text(value='Explain Python?', description='Question:', placeholder='Type your question here')

Button(description='Ask', icon='check', style=ButtonStyle(), tooltip='Ask the question')

Output()

In [ ]:
!pip install neo4j langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00


In [ ]:
from IPython.display import display
import ipywidgets as widgets
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, ServiceContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import warnings
import os
from neo4j import GraphDatabase
import spacy

warnings.filterwarnings('ignore')

# ---- NEO4J SETUP ----
neo4j_uri = "neo4j+s://85f2bc34.databases.neo4j.io"
neo4j_user = "neo4j"
neo4j_password = "XnG9jBrff3Ans9W_vCMZ6nQ4TiligdVcs23nGbvMifc"
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [ ]:
# ---- ENVIRONMENT VARIABLES ----
os.environ["GROQ_API_KEY"] = "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"
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# ---- PROMPT TEMPLATE ----
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Graph Insights: {graph_insights}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information and graph insights, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

In [ ]:
pip install llama-index pymupdf


In [ ]:
import spacy
from llama_index.readers.file import PyMuPDFReader

# Define the context for your prompt
context = "This document provides technical content for AeroCraft ACE-900 systems."

# Load the PDF
pdf_path = "/content/Manual example - AeroCraft ACE-900.pdf"
reader = PyMuPDFReader()
documents = reader.load_data(file_path=pdf_path)  #correct keyword: file_path

#Load SpaCy model
nlp = spacy.load("en_core_web_sm")


In [ ]:
from neo4j import GraphDatabase
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Efficient batch entity and relation extractor
def extract_concept_pairs(documents):
    pairs = []
    doc_texts = [doc.text for doc in documents]

    # Process all documents in batch
    for nlp_doc in nlp.pipe(doc_texts, batch_size=10):
        concepts = [ent.text for ent in nlp_doc.ents if ent.label_ in ("ORG", "PRODUCT")]
        for i in range(len(concepts) - 1):
            pairs.append((concepts[i], concepts[i + 1]))

    return pairs

# Efficient batch graph population in Neo4j
def populate_graph_batch(documents, driver):
    concept_pairs = extract_concept_pairs(documents)

    with driver.session() as session:
        session.run(
            """
            UNWIND $pairs AS pair
            MERGE (c1:Concept {name: pair[0]})
            MERGE (c2:Concept {name: pair[1]})
            MERGE (c1)-[:RELATED_TO]->(c2)
            """,
            pairs=concept_pairs
        )

# Example call (after loading your documents and setting up driver)
populate_graph_batch(documents, driver)
# print("Graph populated productively!")


In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.indices.vector_store import VectorStoreIndex
from llama_index.core.storage import StorageContext
from llama_index.core.indices.loading import load_index_from_storage  # ✅ Corrected import

# Split the documents into nodes
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

# Set up embedding model and LLM
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

# Set global settings
Settings.embed_model = embed_model
Settings.llm = llm

# Create vector store index
vector_index = VectorStoreIndex.from_documents(
    documents,
    show_progress=True,
    node_parser=text_splitter
)

# Persist to disk
vector_index.storage_context.persist(persist_dir="./storage_mini")

# Load the index from storage
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")
index = load_index_from_storage(storage_context)


Parsing nodes:   0%|          | 0/574 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/574 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/618 [00:00<?, ?it/s]

In [ ]:
# Create the interactive widgets
input_box = widgets.Text(
    value='Explain Python?',
    placeholder='Type your question here',
    description='Question:',
    disabled=False
)

output_area = widgets.Output()

In [ ]:
#Query Enhancement with Neo4j

def get_graph_insights(question):
  with driver.session() as session:
    result = session.run(
         """
            MATCH (c:Concept)
            WHERE toLower(c.name) CONTAINS toLower($question)
            OPTIONAL MATCH (c)-[r:RELATED_TO]->(other:Concept)
            RETURN c.name AS concept, collect(other.name) AS related_concepts
            """,
         question=question
         )
    insights = []
    for record in result:
       insights.append(f"Concept: {record['concept']}, Related Concepts: {', '.join(record['related_concepts'])}")
       return "\n".join(insights) if insights else "No relevant graph insights found."

In [ ]:
def on_button_click(b):
  with output_area:
    output_area.clear_output()
    question = input_box.value
    graph_insights = get_graph_insights(question)
    query_prompt = prompt_template.format(context=context, graph_insights=graph_insights, question=question)
    resp = query_engine.query(query_prompt)
    print(resp.response)


button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

button.on_click(on_button_click)

display(input_box, button, output_area)

#Query Engine Setup
# query_engine = index.as_query_engine(service_context=service_context)
query_engine = index.as_query_engine()


Text(value='Explain Python?', description='Question:', placeholder='Type your question here')

Button(description='Ask', icon='check', style=ButtonStyle(), tooltip='Ask the question')

Output()

In [ ]:
button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

button.on_click(on_button_click)

display(input_box, button, output_area)

# ---- QUERY ENGINE SETUP ----
# query_engine = index.as_query_engine(service_context=service_context)
query_engine = index.as_query_engine()


Text(value='what all subsystems are in the electrical system?', description='Question:', placeholder='Type you…

Button(description='Ask', icon='check', style=ButtonStyle(), tooltip='Ask the question')

Output()

In [ ]:
!pip install streamlit pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st

st.title("Streamlit App from Colab")
st.write("This app is running from Google Colab using ngrok.")


Writing app.py


In [ ]:
!ngrok authtoken 2woOR1776Qi2xJmdaH47DdhWI0A_4bLRYGDzDJmy641Q1h1C6

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!ngrok http 8501 -header "ngrok-skip-browser-warning: true"


ERROR:  unknown shorthand flag: 'e' in -eader
